# Análisis del perfil turístico que visitó Gran Canaria 2022

![Portada](https://www.holaislascanarias.com/sites/default/files/block-images/2021-06/Maspalomas.jpg)

El objetivo de este análisis es comprender las características demográficas, comportamientos, preferencias y patrones de los turistas en periodo de postpandemia, utilizando como base la encuesta de gasto turístico realizada por el Instituto Nacional de estadísticas (ISTAC).

A lo largo del análisis se emplean técnicas de exploración de datos (EDA) y herramientas de visualización desarrolladas en Python.

# **1.Preparación de los datos**


**Imports**
Se importan las librerias, módulos y funciones que vamos a usar a lo largo del análisis.

In [60]:
#Librerias que vamos a usar a lo largo del EDA
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import stats, f_oneway, chi2_contingency
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import sys
sys.path.append('utils')
import utils as funciones

# **1.2 Carga de la base de datos y visualización inicial**

Cargamos el dataset con el que vamos a trabajar

In [61]:
#Llamamos al csv y creamos el df de perfil turistico
df_perfil_turistico = pd.read_csv("./data/Perfil_turístico.csv", sep=";", encoding="latin1")

In [62]:
#Observamos sus primeras columnas
df_perfil_turistico.head()

,Sexo,Edad,Nacionalidad,Viaja_solo,Viaja_pareja,Viaja_hijo,Viaja_amigos,Viaja_otros,Noches,Tipo_alojamiento,...,Visitas_totales_Islas,Visitas_total_GC,Calificacion,Repetiria,Nivel_educativo,Situacion_laboral,Ingresos,Personas_hogar,Cuatrimestre,Año
0,6,42,España,No,No,No,No,Si,5,3,...,2,2,4,10,4,1,1,1,Q1,2019
1,1,27,España,No,Si,No,No,No,4,4,...,10,3,3,10,4,1,2,2,Q1,2019
2,6,63,Italia,Si,No,No,No,No,6,3,...,1,1,1,1,4,2,1,1,Q1,2019
3,6,43,Reino Unido,No,No,Si,No,No,6,4,...,1,1,4,9,4,5,3,4,Q1,2019
4,1,43,España,No,No,No,Si,No,4,4,...,3,2,4,10,4,1,2,1,Q1,2019


# **1.3 Limpieza de los datos**

Hacemos el tratamiento de limpieza de los datos que sea necesario para su posterior uso.

In [63]:
#Limpieza de variables categóticas
# Se sabe que en la variable sexo 1=Hombre y 6=Mujer por lo que vamos a hacer la transformacion
df_perfil_turistico["Sexo"] = df_perfil_turistico["Sexo"].replace({1: "Hombre", 6: "Mujer"})

In [64]:
#Nivel educativo
df_perfil_turistico["Nivel_educativo"] = df_perfil_turistico["Nivel_educativo"].replace({1:"Sin estudios", 2:"Estudios primarios", 3:"Estudios secundarios", 4:"Estudios superiores"})

In [65]:
#Situacion laboral
df_perfil_turistico["Situacion_laboral"] = df_perfil_turistico["Situacion_laboral"].replace({ 1:"Asalariado", 2:"Autonomo", 3:"Empresario_peque", 4:"Empresario_grande",
5:"Desempleado", 6:"Estudiante", 7:"Jubilado", 8:"Incapacitado", 9:"Militar", 10:"Dependients", 11:"Otros"})


In [66]:
#Ingresos
df_perfil_turistico["Ingresos"] = df_perfil_turistico["Ingresos"].replace({1:"Menos 25.000", 2:"25.000-49.999",3:"50.000-74.999", 4:"+75.000"})

In [67]:
#Tipo de alojamiento
df_perfil_turistico["Tipo_alojamiento"] = df_perfil_turistico["Tipo_alojamiento"].replace({1:"Hotel 1-2-3*", 2:"Hotel 4*",3:"Hotel 5*", 4:"Apartamento/Villa", 5:"Vivienda/airbnb", 6:"Alojamiento privado", 7:"Alojamiento rural"})

In [68]:
#Repetiría
df_perfil_turistico["Repetiria"] = df_perfil_turistico["Repetiria"].replace({1:"(1)", 2:"(2)",3:"(3)", 4:"(4)", 5:"(5)", 6:"(6)", 7:"(7)",8:"(8)",9:"(9)",10:"(10)"})

In [69]:
#Calificacion
df_perfil_turistico["Calificacion"] = df_perfil_turistico["Calificacion"].replace({1:"Mucho peor de lo esperado", 2:"Peor de lo esperado",3:"A la altura de las expectativas", 4:"Mejor de lo esperado", 5:"Mucho mejor de lo esperado"})

In [70]:
#Creamos una columna con el tipo de aconmpañante
columnas_tipo_acompañante = ["Viaja_solo", "Viaja_pareja", "Viaja_hijo", "Viaja_amigos", "Viaja_otros"]

# Crear la columna "Resultado" solo basándose en esas columnas
df_perfil_turistico["Modo_viaje"] = df_perfil_turistico.apply(lambda row: ', '.join(col for col in columnas_tipo_acompañante if row[col] == "Si"), axis=1)


In [71]:
#Eliminamos la columna Tipo de viaje
df_perfil_turistico = df_perfil_turistico.drop(columns=["Viaja_pareja","Viaja_hijo","Viaja_otros","Viaja_solo","Viaja_amigos"])

In [72]:
df_perfil_turistico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12295 entries, 0 to 12294
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Sexo                     12295 non-null  object 
 1   Edad                     12295 non-null  int64  
 2   Nacionalidad             12295 non-null  object 
 3   Noches                   12295 non-null  int64  
 4   Tipo_alojamiento         12295 non-null  object 
 5   Coste_vuelos             12295 non-null  float64
 6   Coste_alojamiento        12295 non-null  float64
 7   Gastos                   12295 non-null  float64
 8   Horas_fuera_alojamiento  12295 non-null  int64  
 9   Visitas_totales_Islas    12295 non-null  int64  
 10  Visitas_total_GC         12295 non-null  int64  
 11  Calificacion             12295 non-null  object 
 12  Repetiria                12295 non-null  object 
 13  Nivel_educativo          12295 non-null  object 
 14  Situacion_laboral     

In [73]:
# Convertir algunas columnas a float
df_perfil_turistico['Edad'] = df_perfil_turistico['Edad'].astype(float)
df_perfil_turistico['Noches'] = df_perfil_turistico['Noches'].astype(float)
df_perfil_turistico['Personas_hogar'] = df_perfil_turistico['Personas_hogar'].astype(float)
# Verificar el cambio
df_perfil_turistico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12295 entries, 0 to 12294
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Sexo                     12295 non-null  object 
 1   Edad                     12295 non-null  float64
 2   Nacionalidad             12295 non-null  object 
 3   Noches                   12295 non-null  float64
 4   Tipo_alojamiento         12295 non-null  object 
 5   Coste_vuelos             12295 non-null  float64
 6   Coste_alojamiento        12295 non-null  float64
 7   Gastos                   12295 non-null  float64
 8   Horas_fuera_alojamiento  12295 non-null  int64  
 9   Visitas_totales_Islas    12295 non-null  int64  
 10  Visitas_total_GC         12295 non-null  int64  
 11  Calificacion             12295 non-null  object 
 12  Repetiria                12295 non-null  object 
 13  Nivel_educativo          12295 non-null  object 
 14  Situacion_laboral     

In [74]:
#¿Valores duplicados?
df_perfil_turistico.duplicated(keep=False)
len(df_perfil_turistico[df_perfil_turistico.duplicated(keep = False)]) # keep = "first", keep = False

0

In [75]:
#¿Valores nulos?
df_perfil_turistico.isnull()
df_perfil_turistico.isnull().sum()

Sexo                       0
Edad                       0
Nacionalidad               0
Noches                     0
Tipo_alojamiento           0
Coste_vuelos               0
Coste_alojamiento          0
Gastos                     0
Horas_fuera_alojamiento    0
Visitas_totales_Islas      0
Visitas_total_GC           0
Calificacion               0
Repetiria                  0
Nivel_educativo            0
Situacion_laboral          0
Ingresos                   0
Personas_hogar             0
Cuatrimestre               0
Año                        0
Modo_viaje                 0
dtype: int64

In [76]:
#¿Outliers?
variables_numericas=["Edad","Noches","Coste_vuelos","Coste_alojamiento","Gastos","Horas_fuera_alojamiento","Visitas_totales_Islas","Visitas_total_GC","Personas_hogar"]

In [77]:
# Calcular límites inferior y superior
outliers = {}
for var in variables_numericas:
    q1 = df_perfil_turistico[var].quantile(0.25)
    q3 = df_perfil_turistico[var].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Identificar outliers
    outliers[var] = df_perfil_turistico[(df_perfil_turistico[var] < lower_bound) | (df_perfil_turistico[var] > upper_bound)]

    #print(f"Outliers en {var}:\n{outliers[var]}")

Reemplazar outliers con la mediana ya que mantiene la tendencia central del conjunto de datos si que los valores extremos sesgen los resultados


In [78]:
# Reemplazar valores fuera del rango por la mediana
for var in variables_numericas:
    q1 = df_perfil_turistico[var].quantile(0.25)
    q3 = df_perfil_turistico[var].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    median = df_perfil_turistico[var].median()
    df_perfil_turistico[var] = df_perfil_turistico[var].apply(lambda x: median if (x < lower_bound or x > upper_bound) else x)

In [79]:
#Vemos el total de columnas
df_perfil_turistico.columns

Index(['Sexo', 'Edad', 'Nacionalidad', 'Noches', 'Tipo_alojamiento',
       'Coste_vuelos', 'Coste_alojamiento', 'Gastos',
       'Horas_fuera_alojamiento', 'Visitas_totales_Islas', 'Visitas_total_GC',
       'Calificacion', 'Repetiria', 'Nivel_educativo', 'Situacion_laboral',
       'Ingresos', 'Personas_hogar', 'Cuatrimestre', 'Año', 'Modo_viaje'],
      dtype='object')

In [80]:
df_perfil_turistico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12295 entries, 0 to 12294
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Sexo                     12295 non-null  object 
 1   Edad                     12295 non-null  float64
 2   Nacionalidad             12295 non-null  object 
 3   Noches                   12295 non-null  float64
 4   Tipo_alojamiento         12295 non-null  object 
 5   Coste_vuelos             12295 non-null  float64
 6   Coste_alojamiento        12295 non-null  float64
 7   Gastos                   12295 non-null  float64
 8   Horas_fuera_alojamiento  12295 non-null  float64
 9   Visitas_totales_Islas    12295 non-null  float64
 10  Visitas_total_GC         12295 non-null  float64
 11  Calificacion             12295 non-null  object 
 12  Repetiria                12295 non-null  object 
 13  Nivel_educativo          12295 non-null  object 
 14  Situacion_laboral     

Los datos ya han sido limpiados y están listos para comenzar con el análisis.

Comencemos por explorar la tipología de las variables, definiendo su cardinalidad para proceder con su posterior clasificación

In [81]:
# Calculemos ahora su cardinalidad
cardinalidad = round((df_perfil_turistico.nunique() /len (df_perfil_turistico))*100,2)
cardinalidad


Sexo                        0.02
Edad                        0.58
Nacionalidad                0.11
Noches                      0.10
Tipo_alojamiento            0.06
Coste_vuelos               18.28
Coste_alojamiento          24.14
Gastos                     16.03
Horas_fuera_alojamiento     0.16
Visitas_totales_Islas       0.14
Visitas_total_GC            0.10
Calificacion                0.04
Repetiria                   0.09
Nivel_educativo             0.03
Situacion_laboral           0.09
Ingresos                    0.03
Personas_hogar              0.03
Cuatrimestre                0.03
Año                         0.02
Modo_viaje                  0.04
dtype: float64

In [82]:
# Calculemos ahora su cardinalidad
funciones.cardi(df_perfil_turistico,15,65)

,Cardi,% Cardi,Clasificacion
Sexo,2,0.016267,Binaria
Edad,71,0.577471,Numerica Discreta
Nacionalidad,14,0.113867,Categorica
Noches,12,0.097601,Categorica
Tipo_alojamiento,7,0.056934,Categorica
Coste_vuelos,2248,18.283855,Numerica Discreta
Coste_alojamiento,2968,24.139894,Numerica Discreta
Gastos,1971,16.030907,Numerica Discreta
Horas_fuera_alojamiento,20,0.162668,Numerica Discreta
Visitas_totales_Islas,17,0.138268,Numerica Discreta


# **2. Análisis**

Una vez determinada la cardinalidad, podemos comprender mejor las variables y definirlas en la siguiente tabla. Esta tabla nos servirá como guía para identificar cada variable y aplicar el análisis adecuado a cada una de ellas.

|Variable|Descripción|Tipo_de_Variable|Importancia inicial|Nota|
|-|-|-|-|-|
|Sexo| Hombre o mujer|Binaria|1|Explica análisis demográfico|
|Edad| Edad del encuestado|Numérica Discreta|1|Explica análisis demográfico|
|Nacionalidad|Nacionalidad del turista|Categórica|1|Explica análisis demográfico|
|Noches|Número de noches que pernoctó en la isla|Númerica Discreta|1|Define el comportamiento del gasto|
|Tipo_alojamiento| Tipo de alojamiento que se hospedó|Categórica|1|Define el comportamiento del gasto|
|Coste_vuelos|Coste en euros del billete de avión|Numérica Discreta|1|Define el comportamiento del gasto|
|Coste_alojamiento|Coste en euros de las noches alojadas|Numérica Discreta|1|Define el comportamiento del gasto|
|Gasto| Gasto del turista en su estancia en la isla|Numérica Discreta|1|Define el comportamiento del gasto|
|Horas_fuera_alojamiento| Horas fuera del alojamiento|Numérica discreta|3|Comportamiento durante la estancia|
|Visitas_totales_Islas| Número de veces que ha visitados las Islas Canarias|Numérica Discreta|2|Comportamiento durante la estancia|
|Visitas_total_GC| Número de veces que ha estado en Gran Canaria|Numérica Discreta|2|Comportamiento durante la estancia|
|Calificacion| Definida en 5 categorías|Categórica|2|Comportamiento durante la estancia|
|Repetiria| Del 1-10 el valor de repetición|Categórica|2|Comportamiento durante la estancia|
|Nivel_educativo| Nivel educativo|Categórica|1|Explica análisis demográfico|
|Situacion_laboral| Situación laboral|Categórica|1|Explica análisis demográfico|
|Ingresos| Ingresos por tramos, del turista|Categórica|1|Explica análisis demográfico|
|Personas_hogar| Número de personas del hogar |Numérica Discreta|1|Explica análisis demográfico|
|Cuatrimestre| Cuatrimestre de estudio|Categórica|1|Temporadas del estudio|
|Año| 2019 -2022|Binaria|1|Define los datos antes y después de pandemia|
|Modo viaje| Tipo de acompañante|Categórica|1|Define el comportamiento del gasto|